In [2]:
# %load pso.py
import math
import numpy as np
from numba import jit
from failed_exec import create_n_particles
from evaluate import evaluate, update_velocity, update_position

@jit(parallel=True)
def error(position):
    err = 0.0
    for i in range(len(position)):
    
        xi = position[0][i]
        err += (xi * xi) - (10 * np.cos(2 * np.pi * xi)) + 10
    return err

@jit(parallel=True)
def PSO(costFunc,bounds,maxiter, swarm_init):

    num_dimensions=len(swarm_init[0])
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    num_particles = len(swarm_init)
    # establish the swarm
    swarm = create_n_particles(num_particles, num_dimensions, swarm_init)
    # begin optimization loop
    i=0
    while i < maxiter:
    
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j]['pos_best_i'], swarm[j]['err_best_i']  = evaluate(costFunc, swarm[j])
    
            # determine if current particle is the best (globally)
            if swarm[j]['err_i'] < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j]['position_i'])
                err_best_g=float(swarm[j]['err_i'])

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j]['velocity_i'] = update_velocity(pos_best_g, swarm[j])
            swarm[j]['position_i'] = update_position(bounds, swarm[j])
        i+=1

    return pos_best_g, err_best_g

if __name__ == '__main__':
    num_particles = 15
    swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
    bounds=[(-10,10),(-10,10)]
    PSO(error,bounds,maxiter=3, swarm_init=swarm_init)
    print('working')
#     particles = create_n_particles(100, 2, bounds)
#     #evaluate(particles[0])
#     pos_best_g = particles[0]['position_i']
#     update_velocity(pos_best_g, particles[1])
#     update_position(bounds, particles[1])
    

working


In [ ]:
# %load evaluate.py
from failed_exec import create_n_particles
from numba import jit
import math
import random

@jit(debug=True)
def error(position):
    err = 0.0
    for i in range(len(position)):
    
        xi = position[0][i]
        err += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
    return err

@jit
def evaluate(costFunc, p):
    p['err_i'] = costFunc(p['position_i'])
    # check to see if the current position is an individual best
    if p['err_i'] < p['err_best_i'] or p['err_best_i']==-1:
        p['pos_best_i'] = p['position_i']
        p['err_best_i'] = p['err_i']
    return p['pos_best_i'], p['err_best_i']

@jit
def update_velocity(pos_best_g, p):
    w=0.5       # constant inertia weight (how much to weigh the previous velocity)
    c1=1        # cognative constant
    c2=2        # social constant

    for i in range(0, p['num_dimensions']):
        r1=random.random()
        r2=random.random()

        vel_cognitive=c1*r1*(p['pos_best_i'][0][i]-p['position_i'][0][i])
        vel_social=c2*r2*(pos_best_g[0][i]-p['position_i'][0][i])
        p['velocity_i'][0][i]=w*p['velocity_i'][0][i]+vel_cognitive+vel_social
    return p['velocity_i']

@jit
def update_position(bounds, p):
    for i in range(0, p['num_dimensions']):
        p['position_i'][0][i]=p['position_i'][0][i]+p['velocity_i'][0][i]

        # adjust maximum position if necessary
        if p['position_i'][0][i]>bounds[i][1]:
            p['position_i'][0][i]=bounds[i][1]

        # adjust minimum position if neseccary
        if p['position_i'][0][i] < bounds[i][0]:
            p['position_i'][0][i]=bounds[i][0]
    return p['position_i']



if __name__ == '__main__':
    bounds=[(-10,10),(-10,10)]
    particles = create_n_particles(100, 2, bounds)
#     evaluate(error, particles[0])
    pos_best_g = particles[0]['position_i']
    update_velocity(pos_best_g, particles[1])
#     update_position(bounds, particles[1])

In [3]:
from pso import PSO, error, evaluate, update_velocity, update_position
from failed_exec import create_n_particles

bounds=[(-10,10),(-10,10)]
particles = create_n_particles(100, 2, bounds)

In [4]:
eval_time = %%timeit -o evaluate(error, particles[0])

16.4 µs ± 900 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
%load_ext line_profiler

In [6]:
%prun evaluate(error,particles[2])

In [7]:
from particle import Particle
import numpy as np
num_particles = 15
swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
bounds=[(-10,10),(-10,10)]

particle = Particle(swarm_init[0], len(swarm_init[0]))

In [9]:
import math
def error(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return err

In [10]:
new_time = %%timeit -o particle.evaluate(error)

2.04 µs ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
# %load particle.py
#dependencies
import random
import math
import copy # for array copying
import sys

class Particle:
    def __init__(self,x0, num_dimensions):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        self.num_dimensions = num_dimensions
        
        for i in range(0, self.num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0, self.num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

In [7]:
%%writefile jitParticle.py
import numba
import random
import math
import copy # for array copying
import sys
import numpy as np
from numba import jitclass, jit
from numba import int32, float32, int64

spec = [
    ('num_dimensions', int32),
    ('position_i', float32[:]),         # particle position
    ('velocity_i', float32[:]),
    ('pos_best_i', float32[:]),
    ('err_best_i', float32),
    ('err_i', float32)        
]

@jitclass(spec)
class Particle:
    def __init__(self, x0, num_dimensions):
        
        self.num_dimensions = num_dimensions
        self.position_i = np.zeros((1,self.num_dimensions), dtype=np.float32)          # particle position
        self.velocity_i = np.zeros((1,self.num_dimensions), dtype=np.float32)          # particle velocity
        self.pos_best_i = np.zeros((1,self.num_dimensions), dtype=np.float32)         # best position individual
        self.err_best_i = -1          # best error individual
        self.err_i = -1               # error individual
        
        for i in range(0, self.num_dimensions):
            self.velocity_i[i] = random.uniform(-1,1)
            self.position_i[i] = x0[i]

    # evaluate current fitness
    def evaluate(self):
        
        self.err_i = error(self.position_i)
        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i = self.position_i
            self.err_best_i = self.err_i

    # update new particle velocity
    def update_velocity(self, pos_best_g):
        w = 0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1 = 1        # cognative constant
        c2 = 2        # social constant

        for i in range(0, self.num_dimensions):
            r1 = random.random()
            r2 = random.random()

            vel_cognitive = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            vel_social = c2 * r2 * (pos_best_g[i] - self.position_i[i])
            self.velocity_i[i] = w * self.velocity_i[i] + vel_cognitive + vel_social

    # update the particle position based off new velocity updates
    def update_position(self, bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]


@jit(parallel=True)
def error(position):
    err = 0.0
    for i in range(len(position)):

        xi = position[0][i]
        err += (xi * xi) - (10 * np.cos(2 * np.pi * xi)) + 10
    return err

Overwriting jitParticle.py


In [2]:
from jitParticle import Particle

In [2]:
def PSO(costFunc,bounds,maxiter, swarm_init):
    global num_dimensions

    num_dimensions=len(swarm_init[0])
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    num_particles = len(swarm_init)
    # establish the swarm
    swarm = [Particle(position, num_dimensions) for position in swarm_init]
    # begin optimization loop
    i=0
    while i < maxiter:
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1

    # print final results
    #print ('\n')
    #print (pos_best_g,' , ', err_best_g)
    return pos_best_g, err_best_g

In [3]:
import math
from numba import njit
@njit(parallel=True)
def error(position):
    err = 0.0
    for i in range(len(position)):
    
        xi = position[0][i]
        err += (xi * xi) - (10 * np.cos(2 * np.pi * xi)) + 10
    return err

In [4]:
import numpy as np

In [7]:
%time
bounds=[[-10,10],[-10,10]]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
M = 5
subswarm_bests = []
num_particles = 15
for i in range(M):
    #initial= np.random.uniform(-10,10, 2)               # initial starting location [x1,x2...] 
    swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
    subswarm_best,_ = PSO(error,bounds,maxiter=30, swarm_init=swarm_init)
    subswarm_bests.append(subswarm_best)
best_position, best_error = PSO(error, bounds, maxiter=30, swarm_init=subswarm_bests)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


NameError: name 'error' is not defined

In [3]:
import numpy as np
p = Particle(np.random.uniform(-10,10, 2),len(np.random.uniform(-10,10, 2)))

In [4]:
p.evaluate()

TypingError: Failed at nopython (nopython frontend)
Failed at nopython (nopython frontend)
Invalid usage of type(CPUDispatcher(<function error at 0x7fa2b3e28268>)) with parameters (array(float32, 1d, A))
 * parameterized
File "jitParticle.py", line 37
[1] During: resolving callee type: type(CPUDispatcher(<function error at 0x7fa2b3e28268>))
[2] During: typing of call at /home/shubham/Documents/Github/parallelGSO/jitParticle.py (37)
[3] During: resolving callee type: BoundFunction((<class 'numba.types.misc.ClassInstanceType'>, 'evaluate') for instance.jitclass.Particle#5579d835e2f8<num_dimensions:int64,position_i:array(float32, 1d, A),velocity_i:array(float32, 1d, A),pos_best_i:array(float32, 1d, A),err_best_i:float32,err_i:float32>)
[4] During: typing of call at <string> (3)

In [6]:
# %load failed_exec.py
from numba import jit
import numpy as np

# particle_dtypes = np.dtype({'names':['position_i', 'velocity_i', 'pos_best_i',
#                                       'err_best_i', 'err_i', 'num_dimensions'],
#                              'formats':[np.double,np.double, np.double, np.double, np.double, np.int32]
#                             })

@jit
def create_n_particles(n, num_dimensions, swarm_init):
    
    particle_dtypes = np.dtype({'names':['position_i', 'velocity_i', 'pos_best_i',
                                      'err_best_i', 'err_i', 'num_dimensions'],
                                'formats':[(np.double,(1,(num_dimensions))),
                                       (np.double,(1,(num_dimensions))),
                                       (np.double,(1,(num_dimensions))),
                                       np.double, np.double, np.int32]
                            })
    particles = np.empty(n, dtype=particle_dtypes)
    for p,x0 in zip(particles, swarm_init):
        p['err_best_i'] = -1
        p['err_i'] = -1
        p['num_dimensions'] = num_dimensions
        for i in range(num_dimensions):
            p['velocity_i'][0][i] = np.random.uniform(-1,1)
            p['position_i'][0][i] = x0[i]
    return particles

if __name__ == '__main__':
    #swarm_init = [(5, 5) for _ in range(100)]
    num_particles = 15
    swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
    particles = create_n_particles(100, 2, swarm_init)
    print(particles[:3])

[([[3.70029523, 1.55051433]], [[-0.78824256, -0.25742554]], [[ 6.46572286e+170,  8.27255353e-315]], -1., -1., 2)
 ([[8.24497879, 1.00107085]], [[ 0.0113357 , -0.75347892]], [[-1.75613664e+118, -1.75803375e+000]], -1., -1., 2)
 ([[6.12675437, 9.5399259 ]], [[-0.27174346,  0.23809424]], [[ 9.88131292e-324,  0.00000000e+000]], -1., -1., 2)]


In [13]:
# %load pso_fail.py


from numba import jit
import numpy as np
import math

@jit(parallel=True)
def error(position):
    err = 0.0
    for i in range(len(position)):
    
        xi = position[0][i]
        err += (xi * xi) - (10 * np.cos(2 * np.pi * xi)) + 10
    return err

@jit
def PSO(costFunc,bounds,maxiter, swarm_init):

    num_dimensions=len(swarm_init[0])
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    num_particles = len(swarm_init)
    # establish the swarm
    swarm = create_n_particles(num_particles, num_dimensions, swarm_init)
    # begin optimization loop
    i=0
    while i < maxiter:
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j]['pos_best_i'], swarm[j]['err_best_i']  = evaluate(costFunc, swarm[j])

            # determine if current particle is the best (globally)
            if swarm[j]['err_i'] < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j]['position_i'])
                err_best_g=float(swarm[j]['err_i'])

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j]['velocity_i'] = update_velocity(pos_best_g, swarm[j])
            swarm[j]['position_i'] = update_position(bounds, swarm[j])
        i+=1

    # print final results
    #print ('\n')
    #print (pos_best_g,' , ', err_best_g)
    return pos_best_g, err_best_g

bounds = [[-10,10],[-10,10]]
PSO(error,bounds,maxiter=30, swarm_init=swarm_init)
#numba_time =  %%timeit -o PSO(error,bounds,maxiter=30, swarm_init=swarm_init)

NameError: global name 'evaluate' is not defined

In [9]:
from pso_fail import PSO

NameError: name 'jit' is not defined